# Datacube Exploration

This notebook demonstrates how to combine multiple CMIP datasets into a unified datacube and explore the results.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

# Add the scripts directory to the path so we can import our modules
sys.path.append('../scripts')
from cmip_processor import CMIPProcessor, AggregationMethod
from datacube_builder import DatacubeBuilder, InterpolationMethod

## Load CMIP Datasets

First, let's load the available CMIP datasets.

In [ ]:
# Paths to the CMIP data files
data_files = {
    "rcp45": "./data/macav2metdata_huss_CCSM4_r6i1p1_rcp45_2021_2025_CONUS_monthly.nc",
    "rcp85": "./data/macav2metdata_huss_CCSM4_r6i1p1_rcp85_2021_2025_CONUS_monthly.nc"
}

# Import configuration if available
try:
    # Add parent directory to path to import config
    sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
    import config
    # Override paths with config paths if available
    if hasattr(config, 'CMIP_CONFIG') and hasattr(config, 'DATA_DIR'):
        model = config.CMIP_CONFIG.get("models", ["CCSM4"])[0]
        for scenario in config.CMIP_CONFIG.get("scenarios", ["rcp45", "rcp85"]):
            for variable in config.CMIP_CONFIG.get("variables", ["huss"]):
                years = config.CMIP_CONFIG.get("years", "2021_2025")
                region = config.CMIP_CONFIG.get("region", "CONUS")
                frequency = config.CMIP_CONFIG.get("frequency", "monthly")
                file_name = f"macav2metdata_{variable}_{model}_r6i1p1_{scenario}_{years}_{region}_{frequency}.nc"
                file_path = os.path.join(config.DATA_DIR, file_name)
                if scenario in data_files:
                    data_files[scenario] = file_path
except ImportError:
    print("Configuration module not found. Using default paths.")

# Check for missing data files
available_files = {}
missing_files = []
for scenario, file_path in data_files.items():
    if os.path.exists(file_path):
        available_files[scenario] = file_path
    else:
        missing_files.append(file_path)

if not available_files:
    raise FileNotFoundError(
        f"No CMIP data files found. Please place at least one of the following files in the data directory:\n"
        f"- {', '.join(missing_files)}\n"
        f"or update the file paths to point to your data files."
    )
elif missing_files:
    print(f"Warning: Some data files are missing: {', '.join(missing_files)}")
    print(f"Proceeding with available files: {', '.join(available_files.keys())}")

# Helper function to load datasets with non-standard calendars
def load_dataset_with_calendar_handling(file_path):
    try:
        # Try the standard CMIPProcessor
        processor = CMIPProcessor(file_path)
        return processor
    except ValueError as e:
        if "unable to decode time units" in str(e) or "calendar" in str(e):
            print(f"Warning: Non-standard calendar detected in {file_path}")
            print("Attempting to fix by manually loading the dataset...")
            
            # Override CMIPProcessor._load_dataset method to handle non-standard calendars
            import xarray as xr
            
            try:
                # Try to import cftime
                import cftime
                print("Using cftime for non-standard calendar.")
                dataset = xr.open_dataset(file_path, use_cftime=True)
            except ImportError:
                print("cftime not available. Loading without decoding times.")
                dataset = xr.open_dataset(file_path, decode_times=False)
            
            # Create a processor and manually set the dataset
            processor = CMIPProcessor(file_path)
            processor.dataset = dataset
            return processor
        else:
            # Re-raise if it's a different ValueError
            raise

# Initialize processors for each dataset
processors = {}
for scenario, file_path in available_files.items():
    try:
        processors[scenario] = load_dataset_with_calendar_handling(file_path)
        print(f"Loaded {scenario} dataset with dimensions: {processors[scenario].dataset.dims}")
    except Exception as e:
        print(f"Error loading {scenario} dataset: {e}")
        print(f"Skipping {scenario} dataset")

if not processors:
    raise RuntimeError("Failed to load any datasets. Please ensure at least one valid CMIP data file is available.")

## Compare Datasets

Let's compare basic statistics between the datasets.

In [ ]:
# Compare statistics if we have processors
if not processors:
    print("No processors available for comparison")
else:
    # Get a list of all variables across all processors
    all_vars = set()
    for processor in processors.values():
        all_vars.update(processor.dataset.data_vars)
    
    if not all_vars:
        print("No variables found in datasets")
    else:
        # Use the first variable found in the first processor
        var_name = list(all_vars)[0]
        
        # Compare statistics
        stats = []
        for scenario, processor in processors.items():
            if var_name in processor.dataset.data_vars:
                var = processor.dataset[var_name]
                stats.append({
                    "Scenario": scenario,
                    "Min": float(var.min()),
                    "Max": float(var.max()),
                    "Mean": float(var.mean()),
                    "Std": float(var.std())
                })
        
        # Convert to DataFrame for easy viewing
        if stats:
            stats_df = pd.DataFrame(stats)
            display(stats_df)
        else:
            print(f"Variable {var_name} not found in any processor's dataset")

## Visualize Differences Between Scenarios

Let's create maps to visualize the differences between the RCP4.5 and RCP8.5 scenarios.

In [ ]:
# Check if we have both RCP4.5 and RCP8.5 scenarios
if "rcp45" in processors and "rcp85" in processors:
    # Get common variable name
    common_vars = set(processors["rcp45"].dataset.data_vars).intersection(
        set(processors["rcp85"].dataset.data_vars)
    )
    
    if not common_vars:
        print("No common variables found between the scenarios")
    else:
        var_name = list(common_vars)[0]
        
        # Calculate difference between scenarios for the first time point
        rcp45_data = processors["rcp45"].dataset[var_name].isel(time=0)
        rcp85_data = processors["rcp85"].dataset[var_name].isel(time=0)

        # Check coordinate systems
        print("RCP45 coordinates:", rcp45_data.lat.shape, rcp45_data.lon.shape)
        print("RCP85 coordinates:", rcp85_data.lat.shape, rcp85_data.lon.shape)

        # Ensure they have the same coordinates
        if not (rcp45_data.lat.equals(rcp85_data.lat) and rcp45_data.lon.equals(rcp85_data.lon)):
            print("Warning: Datasets have different coordinates.")
            
            # If dimensions are identical but values differ
            if rcp45_data.lat.shape == rcp85_data.lat.shape and rcp45_data.lon.shape == rcp85_data.lon.shape:
                print("Dimensions match but values differ. Using as-is.")
                diff_data = rcp85_data - rcp45_data
            else:
                # Check which has the coarser resolution
                if len(rcp45_data.lat) <= len(rcp85_data.lat) and len(rcp45_data.lon) <= len(rcp85_data.lon):
                    # RCP45 is coarser, so downsample RCP85
                    print("RCP45 has coarser resolution. Downsampling RCP85...")
                    rcp85_data = rcp85_data.sel(
                        lat=rcp45_data.lat.values,
                        lon=rcp45_data.lon.values,
                        method="nearest"
                    )
                else:
                    # RCP85 is coarser, so downsample RCP45
                    print("RCP85 has coarser resolution. Downsampling RCP45...")
                    rcp45_data = rcp45_data.sel(
                        lat=rcp85_data.lat.values,
                        lon=rcp85_data.lon.values,
                        method="nearest"
                    )
                
                # Calculate difference
                diff_data = rcp85_data - rcp45_data
        else:
            # Calculate difference if coordinates are already the same
            diff_data = rcp85_data - rcp45_data

        # Create a 1x3 subplot for comparison
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # Plot RCP4.5
        rcp45_plot = rcp45_data.plot(ax=axes[0], cmap='viridis')
        axes[0].set_title(f"RCP4.5 {var_name} (First Month)")

        # Plot RCP8.5
        rcp85_plot = rcp85_data.plot(ax=axes[1], cmap='viridis')
        axes[1].set_title(f"RCP8.5 {var_name} (First Month)")

        # Plot Difference
        diff_plot = diff_data.plot(ax=axes[2], cmap='RdBu_r')
        axes[2].set_title(f"Difference (RCP8.5 - RCP4.5)")

        plt.tight_layout()
        plt.show()
else:
    # Get all available scenarios
    available_scenarios = list(processors.keys())
    
    if not available_scenarios:
        print("No scenarios available for visualization")
    else:
        # Use the first scenario
        scenario = available_scenarios[0]
        var_names = list(processors[scenario].dataset.data_vars)
        
        if not var_names:
            print(f"No variables found in {scenario} dataset")
        else:
            var_name = var_names[0]
            
            # Plot just the single available scenario
            fig, ax = plt.subplots(figsize=(12, 8))
            data = processors[scenario].dataset[var_name].isel(time=0)
            data.plot(ax=ax, cmap='viridis')
            ax.set_title(f"{scenario.upper()} {var_name} (First Month)")
            plt.tight_layout()
            plt.show()
            
            print(f"Note: Only {scenario} scenario is available. Cannot calculate differences between scenarios.")

## Process Datasets for Datacube Integration

Now let's process each dataset to prepare for integration into a unified datacube.

In [ ]:
# Define common bucketing parameters
lat_bucket_size = 0.5  # degrees
lon_bucket_size = 0.5  # degrees
time_bucket_size = '1M'  # monthly

# Process each dataset
processed_datasets = {}
for scenario, processor in processors.items():
    try:
        processed_datasets[scenario] = processor.process_to_datacube(
            lat_bucket_size=lat_bucket_size,
            lon_bucket_size=lon_bucket_size,
            time_bucket_size=time_bucket_size,
            spatial_agg_method=AggregationMethod.MEAN,
            temporal_agg_method=AggregationMethod.MEAN
        )
        print(f"Processed {scenario} dataset with new dimensions: {processed_datasets[scenario].dims}")
    except Exception as e:
        print(f"Error processing {scenario} dataset: {e}")
        print(f"Attempting simplified processing for {scenario}...")
        
        try:
            # Calculate stride for lat/lon to approximate the requested bucket sizes
            lat_res, lon_res = processor.get_spatial_resolution()
            lat_stride = max(1, int(lat_bucket_size / lat_res))
            lon_stride = max(1, int(lon_bucket_size / lon_res))
            time_stride = 1  # Monthly is typically the native resolution
            
            print(f"Using strides: lat={lat_stride}, lon={lon_stride}, time={time_stride}")
            
            # Create a coarser resolution dataset using slicing
            simplified = processor.dataset.isel(
                lat=slice(0, None, lat_stride),
                lon=slice(0, None, lon_stride),
                time=slice(0, None, time_stride)
            )
            processed_datasets[scenario] = simplified
            print(f"Simplified processing complete for {scenario}. New dimensions: {simplified.dims}")
        except Exception as nested_e:
            print(f"Failed to process {scenario} dataset even with simplified approach: {nested_e}")
            print(f"Skipping {scenario} dataset")

if not processed_datasets:
    print("No datasets were successfully processed. Cannot continue with datacube building.")
else:
    print(f"Successfully processed {len(processed_datasets)} datasets: {', '.join(processed_datasets.keys())}")

## Build a Unified Datacube

Now, let's use the DatacubeBuilder to combine the processed datasets into a unified datacube.

In [ ]:
# Check if we have processed datasets
if not processed_datasets:
    print("No processed datasets available. Cannot build unified datacube.")
else:
    # Initialize the datacube builder
    builder = DatacubeBuilder()

    # Add the processed datasets
    for scenario, dataset in processed_datasets.items():
        builder.add_dataset(scenario, dataset)

    # Build the unified datacube
    try:
        unified_datacube = builder.build_datacube(
            lat_resolution=lat_bucket_size,
            lon_resolution=lon_bucket_size,
            time_resolution=time_bucket_size,
            interpolation_method=InterpolationMethod.LINEAR
        )
        
        # Print information about the unified datacube
        print("Unified Datacube Information:")
        print(unified_datacube)
    except Exception as e:
        print(f"Error building unified datacube: {e}")
        print("Attempting simplified datacube creation...")
        
        # Try a simpler approach if unified datacube building fails
        try:
            # Create a simplified datacube by concatenating datasets along a new dimension
            datasets_list = list(processed_datasets.values())
            scenarios_list = list(processed_datasets.keys())
            
            if len(datasets_list) > 1:
                # For multiple datasets, concatenate along a new "scenario" dimension
                unified_datacube = xr.concat(datasets_list, dim="scenario")
                unified_datacube = unified_datacube.assign_coords(scenario=scenarios_list)
                print("Created simplified datacube by concatenating along 'scenario' dimension")
            else:
                # For single dataset, just use as is
                unified_datacube = datasets_list[0]
                print(f"Using single '{scenarios_list[0]}' dataset as the unified datacube")
            
            print("Simplified Datacube Information:")
            print(unified_datacube)
        except Exception as nested_e:
            print(f"Failed to create even a simplified datacube: {nested_e}")
            print("Cannot continue with datacube exploration without a unified datacube.")

## Explore the Unified Datacube

Let's explore the unified datacube to see how it combines the different datasets.

In [ ]:
# Check if unified_datacube is defined
if 'unified_datacube' in locals():
    # List all variables in the unified datacube
    print("Variables in the unified datacube:")
    for var_name in unified_datacube.data_vars:
        print(f"  - {var_name}")
else:
    print("No unified datacube available for exploration.")

## Compare Variables Across Scenarios

Let's compare the same variable across different scenarios in the unified datacube.

In [ ]:
# Check if unified_datacube is defined
if 'unified_datacube' not in locals():
    print("No unified datacube available for time series analysis.")
else:
    # Get available variables
    var_names = list(unified_datacube.data_vars)
    
    if not var_names:
        print("No variables available in the unified datacube.")
    else:
        # Check if we have both RCP4.5 and RCP8.5 specific variables
        rcp45_vars = [v for v in var_names if 'rcp45' in v]
        rcp85_vars = [v for v in var_names if 'rcp85' in v]
        
        if rcp45_vars and rcp85_vars:
            # Find a common variable (without the scenario prefix)
            common_var_base = None
            for v45 in rcp45_vars:
                for v85 in rcp85_vars:
                    if v45.replace('rcp45_', '') == v85.replace('rcp85_', ''):
                        common_var_base = v45.replace('rcp45_', '')
                        break
                if common_var_base:
                    break
            
            if common_var_base:
                # Get the full variable names
                rcp45_var = f"rcp45_{common_var_base}"
                rcp85_var = f"rcp85_{common_var_base}"
                
                # Select a specific location for time series analysis
                lat_idx = len(unified_datacube.lat) // 2  # Middle latitude index
                lon_idx = len(unified_datacube.lon) // 2  # Middle longitude index
                
                # Extract lat/lon values
                lat_val = float(unified_datacube.lat[lat_idx])
                lon_val = float(unified_datacube.lon[lon_idx])
                
                # Extract time series for this location
                try:
                    rcp45_series = unified_datacube[rcp45_var].isel(lat=lat_idx, lon=lon_idx)
                    rcp85_series = unified_datacube[rcp85_var].isel(lat=lat_idx, lon=lon_idx)
                    
                    # Plot time series comparison
                    plt.figure(figsize=(12, 6))
                    rcp45_series.plot(label="RCP4.5")
                    rcp85_series.plot(label="RCP8.5", linestyle="--")
                    plt.title(f"{common_var_base} Time Series at Lat: {lat_val:.2f}, Lon: {lon_val:.2f}")
                    plt.xlabel('Time')
                    plt.ylabel(f"{common_var_base} (units)")
                    plt.grid(True)
                    plt.legend()
                    plt.tight_layout()
                    plt.show()
                except Exception as e:
                    print(f"Error extracting or plotting time series: {e}")
                    print("Attempting simplified time series extraction...")
                    
                    try:
                        # Try different indexing approach
                        if 'scenario' in unified_datacube.dims:
                            # For simplified concatenated datacube
                            rcp45_idx = list(unified_datacube.scenario.values).index('rcp45')
                            rcp85_idx = list(unified_datacube.scenario.values).index('rcp85')
                            
                            var_name = list(unified_datacube.data_vars)[0]
                            rcp45_series = unified_datacube[var_name].isel(scenario=rcp45_idx, lat=lat_idx, lon=lon_idx)
                            rcp85_series = unified_datacube[var_name].isel(scenario=rcp85_idx, lat=lat_idx, lon=lon_idx)
                            
                            # Plot time series comparison
                            plt.figure(figsize=(12, 6))
                            rcp45_series.plot(label="RCP4.5")
                            rcp85_series.plot(label="RCP8.5", linestyle="--")
                            plt.title(f"{var_name} Time Series at Lat: {lat_val:.2f}, Lon: {lon_val:.2f}")
                            plt.xlabel('Time')
                            plt.ylabel(f"{var_name} (units)")
                            plt.grid(True)
                            plt.legend()
                            plt.tight_layout()
                            plt.show()
                        else:
                            raise ValueError("No scenario dimension in the datacube")
                    except Exception as nested_e:
                        print(f"Failed to extract time series with alternative method: {nested_e}")
            else:
                print("No common variable found between RCP4.5 and RCP8.5 datasets.")
        else:
            # Handle case with only one scenario
            # Use the first available variable
            var_name = var_names[0]
            
            # Check if 'lat' and 'lon' dimensions exist
            if 'lat' in unified_datacube.dims and 'lon' in unified_datacube.dims:
                lat_idx = len(unified_datacube.lat) // 2  # Middle latitude index
                lon_idx = len(unified_datacube.lon) // 2  # Middle longitude index
                
                # Extract lat/lon values
                lat_val = float(unified_datacube.lat[lat_idx])
                lon_val = float(unified_datacube.lon[lon_idx])
                
                # Extract time series for this location
                try:
                    time_series = unified_datacube[var_name].isel(lat=lat_idx, lon=lon_idx)
                    
                    # Plot time series
                    plt.figure(figsize=(12, 6))
                    time_series.plot()
                    plt.title(f"{var_name} Time Series at Lat: {lat_val:.2f}, Lon: {lon_val:.2f}")
                    plt.xlabel('Time')
                    plt.ylabel(f"{var_name} (units)")
                    plt.grid(True)
                    plt.tight_layout()
                    plt.show()
                except Exception as e:
                    print(f"Error plotting time series: {e}")

## Calculate and Visualize Scenario Differences

Let's calculate the difference between scenarios across the entire spatial domain over time.

In [ ]:
# Check if unified_datacube is defined
if 'unified_datacube' not in locals():
    print("No unified datacube available for calculating scenario differences.")
else:
    # Check if we have both RCP4.5 and RCP8.5 specific variables
    rcp45_vars = [v for v in unified_datacube.data_vars if 'rcp45' in v]
    rcp85_vars = [v for v in unified_datacube.data_vars if 'rcp85' in v]
    
    if not (rcp45_vars and rcp85_vars):
        # Check if we have a scenario dimension instead
        if 'scenario' in unified_datacube.dims and len(unified_datacube.scenario) >= 2:
            # For simplified concatenated datacube
            try:
                rcp45_idx = list(unified_datacube.scenario.values).index('rcp45')
                rcp85_idx = list(unified_datacube.scenario.values).index('rcp85')
                
                var_name = list(unified_datacube.data_vars)[0]
                
                # Calculate difference between scenarios
                scenario_difference = unified_datacube[var_name].isel(scenario=rcp85_idx) - unified_datacube[var_name].isel(scenario=rcp45_idx)
                
                # Plot the spatial pattern of differences for the first time point
                plt.figure(figsize=(12, 8))
                scenario_difference.isel(time=0).plot(cmap='RdBu_r')
                plt.title(f"Difference in {var_name} (RCP8.5 - RCP4.5) at {unified_datacube.time.values[0]}")
                plt.tight_layout()
                plt.show()
                
                # Create a diff dataset for consistency with later code
                diff_dataset = xr.Dataset(
                    data_vars={
                        f"{var_name}_difference": scenario_difference
                    },
                    coords={
                        'lat': unified_datacube.lat,
                        'lon': unified_datacube.lon,
                        'time': unified_datacube.time
                    }
                )
            except Exception as e:
                print(f"Error calculating scenario differences: {e}")
                print("Cannot create difference visualization.")
        else:
            print("Multiple scenarios (RCP4.5 and RCP8.5) are required to calculate differences.")
            print("Only found variables:", list(unified_datacube.data_vars))
    else:
        # Find a common variable (without the scenario prefix)
        common_var_base = None
        for v45 in rcp45_vars:
            for v85 in rcp85_vars:
                if v45.replace('rcp45_', '') == v85.replace('rcp85_', ''):
                    common_var_base = v45.replace('rcp45_', '')
                    break
            if common_var_base:
                break
        
        if common_var_base:
            # Get the full variable names
            rcp45_var = f"rcp45_{common_var_base}"
            rcp85_var = f"rcp85_{common_var_base}"
            
            try:
                # Calculate difference between scenarios
                scenario_difference = unified_datacube[rcp85_var] - unified_datacube[rcp45_var]
                
                # Create a new dataset with the difference
                diff_dataset = xr.Dataset(
                    data_vars={
                        f"{common_var_base}_difference": scenario_difference
                    },
                    coords=unified_datacube.coords
                )
                
                # Plot the spatial pattern of differences for the first time point
                plt.figure(figsize=(12, 8))
                diff_dataset[f"{common_var_base}_difference"].isel(time=0).plot(cmap='RdBu_r')
                plt.title(f"Difference in {common_var_base} (RCP8.5 - RCP4.5) at {unified_datacube.time.values[0]}")
                plt.tight_layout()
                plt.show()
            except Exception as e:
                print(f"Error calculating scenario differences: {e}")
                print("Cannot create difference visualization.")
        else:
            print("No common variable found between RCP4.5 and RCP8.5 datasets.")

## Analyze Temporal Trends

Let's analyze how the difference between scenarios evolves over time.

In [ ]:
# Check if diff_dataset is defined
if 'diff_dataset' not in locals():
    print("No difference dataset available for temporal trend analysis.")
else:
    try:
        # Find the difference variable
        diff_var = [v for v in diff_dataset.data_vars if 'difference' in v]
        if diff_var:
            # Take the first difference variable found
            diff_var = diff_var[0]
            
            # Calculate the spatial mean difference for each time point
            mean_diff_over_time = diff_dataset[diff_var].mean(dim=['lat', 'lon'])
            
            # Plot the trend
            plt.figure(figsize=(12, 6))
            mean_diff_over_time.plot(marker='o')
            plt.title(f"Mean Difference in {diff_var.replace('_difference', '')} (RCP8.5 - RCP4.5) Over Time")
            plt.xlabel('Time')
            plt.ylabel(f"Mean Difference ({diff_var.replace('_difference', '')} units)")
            plt.grid(True)
            plt.tight_layout()
            plt.show()
        else:
            print("No difference variable found in the difference dataset.")
    except Exception as e:
        print(f"Error analyzing temporal trends: {e}")
        print("Cannot create temporal trend visualization.")

## Create a Multi-Variable Analysis

If there are multiple variables in the datasets, let's analyze relationships between them.

In [ ]:
# Check if unified_datacube is defined
if 'unified_datacube' not in locals():
    print("No unified datacube available for multi-variable analysis.")
else:
    # Get all variables
    all_vars = list(unified_datacube.data_vars)
    
    # Check if there are multiple variables
    if len(all_vars) <= 1:
        # For scenario dimension cube
        if 'scenario' in unified_datacube.dims and len(unified_datacube.scenario) >= 2:
            var_name = all_vars[0]
            try:
                # Compare same variable across scenarios
                scenarios = list(unified_datacube.scenario.values)
                if len(scenarios) >= 2:
                    scenario1 = scenarios[0]
                    scenario2 = scenarios[1]
                    
                    # Extract data for a specific time point
                    time_idx = 0
                    var1_data = unified_datacube[var_name].isel(scenario=0, time=time_idx).values.flatten()
                    var2_data = unified_datacube[var_name].isel(scenario=1, time=time_idx).values.flatten()
                    
                    # Remove NaN values
                    mask = ~(np.isnan(var1_data) | np.isnan(var2_data))
                    var1_data = var1_data[mask]
                    var2_data = var2_data[mask]
                    
                    if len(var1_data) > 0 and len(var2_data) > 0:
                        # Create scatter plot
                        plt.figure(figsize=(10, 8))
                        plt.scatter(var1_data, var2_data, alpha=0.5)
                        plt.title(f"Relationship between {var_name} in {scenario1} vs {scenario2}\nat {unified_datacube.time.values[time_idx]}")
                        plt.xlabel(f"{var_name} in {scenario1}")
                        plt.ylabel(f"{var_name} in {scenario2}")
                        plt.grid(True)
                        plt.tight_layout()
                        plt.show()
                        
                        # Calculate correlation
                        corr = np.corrcoef(var1_data, var2_data)[0, 1]
                        print(f"Correlation between {var_name} in {scenario1} and {scenario2}: {corr:.4f}")
                    else:
                        print("Not enough valid data points for analysis after removing NaN values.")
                else:
                    print("Need at least two scenarios for comparison.")
            except Exception as e:
                print(f"Error in multi-variable analysis: {e}")
        else:
            print("Not enough variables for multi-variable analysis. Only found:", all_vars)
    else:
        # Select two variables for comparison
        var1 = all_vars[0]
        var2 = all_vars[1]
        
        try:
            # Extract data for a specific time point
            time_idx = 0
            var1_data = unified_datacube[var1].isel(time=time_idx).values.flatten()
            var2_data = unified_datacube[var2].isel(time=time_idx).values.flatten()
            
            # Remove NaN values
            mask = ~(np.isnan(var1_data) | np.isnan(var2_data))
            var1_data = var1_data[mask]
            var2_data = var2_data[mask]
            
            if len(var1_data) > 0 and len(var2_data) > 0:
                # Create scatter plot
                plt.figure(figsize=(10, 8))
                plt.scatter(var1_data, var2_data, alpha=0.5)
                plt.title(f"Relationship between {var1} and {var2} at {unified_datacube.time.values[time_idx]}")
                plt.xlabel(var1)
                plt.ylabel(var2)
                plt.grid(True)
                plt.tight_layout()
                plt.show()
                
                # Calculate correlation
                corr = np.corrcoef(var1_data, var2_data)[0, 1]
                print(f"Correlation between {var1} and {var2}: {corr:.4f}")
            else:
                print("Not enough valid data points for analysis after removing NaN values.")
        except Exception as e:
            print(f"Error in multi-variable analysis: {e}")

## Save the Unified Datacube

Finally, let's save the unified datacube for future use.

In [ ]:
# Check if we have the necessary datasets to save
if 'unified_datacube' not in locals():
    print("No unified datacube available to save.")
else:
    try:
        # Create output directory if it doesn't exist
        os.makedirs("./data/processed", exist_ok=True)
        
        # Save the unified datacube
        output_path = "./data/processed/unified_cmip_datacube.nc"
        
        if hasattr(builder, 'save_datacube'):
            # If we used the DatacubeBuilder
            builder.save_datacube(output_path)
            print(f"Saved unified datacube to {output_path}")
        else:
            # Direct save
            unified_datacube.to_netcdf(output_path)
            print(f"Saved unified datacube to {output_path}")
            
        # Also save the difference dataset if available
        if 'diff_dataset' in locals():
            try:
                diff_output_path = "./data/processed/scenario_difference_datacube.nc"
                diff_dataset.to_netcdf(diff_output_path)
                print(f"Saved scenario difference dataset to {diff_output_path}")
            except Exception as e:
                print(f"Error saving difference dataset: {e}")
    except Exception as e:
        print(f"Error saving datacube: {e}")

## Conclusion

In this notebook, we've demonstrated how to:

1. Load and process multiple CMIP climate datasets
2. Compare the datasets and visualize their differences
3. Build a unified datacube that combines multiple datasets
4. Explore the unified datacube through various visualizations and analyses
5. Calculate and visualize differences between climate scenarios
6. Save the unified datacube for future use

This approach can be extended to include additional datasets and variables, creating a comprehensive datacube for climate analysis.